In [1]:
import random

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import PIL.Image as pil_image

In [2]:
model = keras.applications.VGG19(include_top=False, input_shape=(45, 45, 3));
model.summary();

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 45, 45, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 45, 45, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 45, 45, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 22, 22, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 22, 22, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 22, 22, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 11, 11, 128)       0     

In [17]:
for layer in model.layers:
    layer.trainable = False

model = keras.Model(inputs=model.inputs, outputs=model.outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 45, 45, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 45, 45, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 45, 45, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 22, 22, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 22, 22, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 22, 22, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 11, 11, 128)       0     

In [4]:
flatten = keras.layers.Flatten()(model.layers[-1].output);
output = keras.layers.Dense(82, activation=keras.activations.softmax)(flatten)

model = keras.Model(inputs = model.inputs, outputs=output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 45, 45, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 45, 45, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 45, 45, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 22, 22, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 22, 22, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 22, 22, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 11, 11, 128)       0   

In [5]:
import os
classifierList = os.listdir('./input/data/extracted_images')
print(classifierList)
print(len(classifierList))

['!', '(', ')', '+', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'alpha', 'ascii_124', 'b', 'beta', 'C', 'cos', 'd', 'Delta', 'div', 'e', 'exists', 'f', 'forall', 'forward_slash', 'G', 'gamma', 'geq', 'gt', 'H', 'i', 'in', 'infty', 'int', 'j', 'k', 'l', 'lambda', 'ldots', 'leq', 'lim', 'log', 'lt', 'M', 'mu', 'N', 'neq', 'o', 'p', 'phi', 'pi', 'pm', 'prime', 'q', 'R', 'rightarrow', 'S', 'sigma', 'sin', 'sqrt', 'sum', 'T', 'tan', 'theta', 'times', 'u', 'v', 'w', 'X', 'y', 'z', '[', ']', '{', '}']
82


In [6]:
print(classifierList[0])

!


In [7]:
def to_one_hot(index):
    results = np.zeros(82)
    results[index] = 1
    return results;

one_hot = []
for i in range (0, 82):
    one_hot.append(to_one_hot(i))

print(len(one_hot))
print(one_hot[0])

82
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
X = []
y = []
testX = []
testy = []

j = 0
k = 0
for i in range (0, 82):
    ImList = os.listdir('./input/data/extracted_images/' + classifierList[i])
    fileDir = './input/data/extracted_images/' + classifierList[i] + '/'
    onehot = one_hot[i]
    k=0

    for str in ImList:
        length = len(ImList)

        image = keras.preprocessing.image.load_img(fileDir + str, target_size=(45, 45))
        image = keras.preprocessing.image.img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = keras.applications.vgg19.preprocess_input(image)

        if k < (length / 2):
            X.append(image)
            y.append(onehot)
        else :
            testX.append(image)
            testy.append(onehot)
        k = k + 1
        j = j + 1
        if j % 1000 == 0:
            print(f'{j}' + "image processed");
            print(len(X))
            print(len(testX))


1000image processed
650
350
2000image processed
1350
650
3000image processed
2350
650
4000image processed
3350
650
5000image processed
4350
650
6000image processed
5350
650
7000image processed
6350
650
8000image processed
7350
650
9000image processed
7797
1203
10000image processed
7797
2203
11000image processed
7797
3203
12000image processed
7797
4203
13000image processed
7797
5203
14000image processed
7797
6203
15000image processed
7797
7203
16000image processed
8203
7797
17000image processed
9203
7797
18000image processed
10203
7797
19000image processed
11203
7797
20000image processed
12203
7797
21000image processed
13203
7797
22000image processed
14203
7797
23000image processed
14975
8025
24000image processed
14975
9025
25000image processed
14975
10025
26000image processed
14975
11025
27000image processed
14975
12025
28000image processed
14975
13025
29000image processed
14975
14025
30000image processed
15026
14974
31000image processed
16026
14974
32000image processed
17026
14974
330

In [9]:
X = np.vstack(X)
print('X Vstack complete')
y = np.vstack(y)
print('Y Vstack complete')

testX = random.sample(testX, 20000);
testy = random.sample(testy, 20000);
print('Random sample complete')

testX = np.vstack(testX)
print('testX Vstack complete')
testy = np.vstack(testy)
print('testY Vstack complete')


X Vstack complete
Y Vstack complete
Random sample complete
testX Vstack complete
testY Vstack complete


In [16]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.fit(X, y, epochs=10)



Epoch 1/10
63/63 [==============================] - 3s 34ms/step - loss: 5.0918 - accuracy: 0.7230
Epoch 2/10
63/63 [==============================] - 2s 24ms/step - loss: 0.0078 - accuracy: 0.9970
Epoch 3/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0056 - accuracy: 0.9980
Epoch 4/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 5/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0043 - accuracy: 0.9995
Epoch 6/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0042 - accuracy: 0.9995
Epoch 7/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 8/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 9/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/10
63/63 [==============================] - 2s 26ms/step - loss: 0.0032 - accuracy: 1.0000


In [11]:
model.evaluate(testX, testy)

 80/625 [==>...........................] - ETA: 12s - loss: 60.7640 - accuracy: 0.0215

KeyboardInterrupt: 

In [ ]:
model.save('model1');